# Lab 1

In [ ]:
from time import sleep
from IPython.display import clear_output
import numpy as np
import heapq

In [ ]:
WALL = "#"
OPEN = "-"
DISPLAY_TIME = 1 # seconds

# FILENAME = "maze1"
# START = (0, 1)
# GOAL = (2, 4)

# FILENAME = "maze2"
# START = (0, 0)
# GOAL = (3, 3)

# FILENAME = "maze3"
# START = (0, 0)
# GOAL = (4, 7)

# FILENAME = "maze4"
# START = (0, 0)
# GOAL = (2, 0)

FILENAME = "maze5"
START = (16, 0)
GOAL = (16, 16)

# FILENAME = "maze6"
# START = (14, 0)
# GOAL = (14, 14)

#### Each position of the maze

In [ ]:
class Node:
    def __init__(self, x, y, g=0, h=0):
        self.x = x
        self.y = y
        self.g = g
        self.h = h
        self.f = g + h
        self.parent = None

    def __lt__(self, other):
        return self.f < other.f
    
    def coordinates(self):
        return (self.x, self.y)


#### Function to read maze from file

In [ ]:
def read_maze_file(filename: str):
    with open(filename + ".txt", "r") as file:
        maze = np.array([list(line.strip()) for line in file])

    if len(maze) == 0:
        raise Exception("Maze file is empty")
    
    for row in maze:
        if len(row) != len(maze[0]):
            raise Exception("Maze file is malformed")
    
    return maze

#### Function to set the start and end positions in the maze

In [ ]:
def position(coordinates: tuple[int, int], maze: np.ndarray):
    x = coordinates[0]
    y = coordinates[1]

    if x < 0 or x >= len(maze) or y < 0 or y >= len(maze[0]) or maze[x][y] == WALL:
        raise ValueError("Invalid position")
    else:
        return Node(x, y)


#### Function to caclulate Heuristic

In [ ]:
def heuristic(a: Node, b: Node):
    return abs(a.x - b.x) + abs(a.y - b.y)

#### Function to print maze

In [ ]:
def print_maze(maze: np.ndarray, checked: np.ndarray, path: list[tuple[int, int]], start: Node, end: Node):
    clear_output()

    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if (i, j) == start.coordinates():
                print("0", end=" ")
            elif (i, j) == end.coordinates():
                print("1", end=" ")
            elif (i, j) in path:
                print("\x1b[33mx\x1b[0m", end=" ")
            elif checked[i][j]:
                print("\x1b[36mo\x1b[0m", end=" ")
            elif maze[i][j] == OPEN:
                print("□", end=" ")
            elif maze[i][j] == WALL:
                print("■", end=" ")
        print()


#### Function to find the path through the maze

In [ ]:
def a_star(maze: np.ndarray, start: Node, goal: Node, display_time = 0.25):
    checked = np.zeros(maze.shape, dtype=bool)
    open = [start]

    while len(open) > 0:
        heapq.heapify(open)
        current = heapq.heappop(open)
        checked[current.x][current.y] = True

        # Print maze at current step
        print_maze(maze, checked, [], start, goal)
        sleep(display_time)

        # # If goal is reached
        if current.x == goal.x and current.y == goal.y:
            path = []
            while current is not None:
                path.append((current.x, current.y))
                current = current.parent

            # Print final maze with path
            print_maze(maze, checked, path[::-1], start, goal)
            return            

        children = []

        # Generate children
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
            node_position = (
                current.x + new_position[0], current.y + new_position[1])

            # If new position is out of bounds
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze) - 1]) - 1) or node_position[1] < 0:
                continue

            # If new position is a wall
            if maze[node_position[0]][node_position[1]] != WALL:
                new_node = Node(
                    node_position[0], node_position[1], current.g, current.h)
                children.append(new_node)

        # Loop through children
        for child in children:
            # If node is already checked
            # if (child.x, child.y) in checked:
            if checked[child.x][child.y]:
                continue

            child.g = current.g + 1
            child.h = heuristic(child, goal)
            child.f = child.g + child.h
            child.parent = current
            # open.append(child)
            heapq.heappush(open, child)
            checked[child.x][child.y] = True


In [ ]:
maze = read_maze_file(FILENAME)
start = position(START, maze)
goal = position(GOAL, maze)

In [ ]:
a_star(maze, start, goal, DISPLAY_TIME)